Enzo Botans, Clara Sachot, Marie Point

# **---- LE JEU DE LA PYRAMIDE ----**
Aussi disponible dans le Git suivant :
https://github.com/marrryzebwjq/Jeu-de-la-Pyramide

## **1 - _Les règles du jeu_**

---

**SUR LA TABLE**

---

- 28 cartes faces cachées en forme de pyramide de 7 étages.

- 4 cartes faces cachées par joueur, qu'ils peuvent chacun observer discrètement en début de jeu.

> Tous les paramètres ci dessus sont personnalisables.

---

**DEROULEMENT DE LA PARTIE**

---

A chaque tour, une carte de la pyramide est retournée en partant du bas.

> Si un joueur possède une carte de même valeur, il peut l'**annoncer** et donner *1 point* à son adversaire. Il peut aussi **bluffer** s'il n'a pas la carte.

> L'adversaire peut l'**accuser de bluff** et si le menteur se fait démasquer, il en prend le *double de points*. Si l'adversaire qui a accusé à tord, l'accusé doit montrer sa carte pour prouver qu'il ne ment pas, et l'accusateur est pénalisé de *3 points*.

La partie se termine lorsque toutes les cartes de la pyramide sont dévoilées. Le gagnant est celui ayant *le moins de points*.

---
**IA**

---

Nous avons créé une IA qui joue le mieux possible contre tout type de joueur, libre à vous d'essayer de la battre.


## **2 - _La création du jeu_**
**Executez les 14 cellules** suivantes en même temps pour générer le support du jeu.

### **_Imports_**

In [ ]:
from copy import copy
import random

from IPython.display import clear_output   # pour clear l'output dans un notebook (.pynb)
#import os                                   # pour clear le terminal (.py)

import matplotlib.pyplot as plt
import numpy as np
from copy import deepcopy

### **Classes : Carte, Deck, Mains des joueurs, Pyramide**

In [2]:
class Carte :
    """ Une carte à jouer ayant une valeur et une couleur.
    Par défaut c'est une carte pour un jeu classique de 52 cartes (A (AS) jusque K (ROI), sans joker).
    Sinon c'est une carte de valeur entre 1 et n.
    """
    VALEURS = {1:"A ", 2:"2 ", 3:"3 ", 4:"4 ", 5:"5 ", 6:"6 ", 7:"7 ", 8:"8 ", 9:"9 ", 10:"10", 11:"J ", 12:"Q ", 13:"K "}
    COULEURS = {"carreau" : "♦", "coeur" : "♥", "pique" : "♠", "trefle" : "♣"}

    def __init__(self, valeur, couleur, custom_val=False, custom_col=False) :
        if not custom_val and valeur not in self.VALEURS :
            raise ValueError(f"Valeur {valeur} invalide.")
        if not custom_col and couleur not in self.COULEURS :
            raise ValueError(f"Couleur {couleur} invalide.")

        self.valeur = valeur
        self.couleur = couleur
        self.custom_val = custom_val
        self.custom_col = custom_col
        self.visible = True

    def __str__(self):
        if self.visible :
            if self.custom_val :
                val = str(self.valeur) + " " if self.valeur < 10 else str(self.valeur)
            else :
                val = self.VALEURS[self.valeur]
            return f"[{val}{self.couleur if self.custom_col else self.COULEURS[self.couleur]}]"
        else :
            return "[ x ]"

    def montrer(self) :
        self.visible = True
        return self

    def cacher(self) :
        self.visible = False
        return self

    def montrer_une_fois(self) :
        c = copy(self.montrer())
        self.cacher()
        return c

    def same_value(self, carte) :
        return self.valeur == carte.valeur

In [3]:
class Deck :
    """ Un tas de cartes.
    """
    def __init__(self, taille=52, custom_cards=False, ui=False) :
        self.ui = ui
        self.taille = taille
        self.custom_cards = custom_cards
        self.cartes = []
        self.creer()

    def __str__(self):
        affichage = ""
        for i in range(len(self.cartes)):
            affichage += self.cartes[i].__str__() + ("\n" if (i+1)%(self.taille//4) == 0 else " ")
        return f"{len(self.cartes)} cartes :\n{affichage}"

    def creer(self) :
        if self.custom_cards :
            self.taille = len(self.custom_cards)
            for c in self.custom_cards :
                if not isinstance(c, Carte) :
                    raise ValueError(f"Carte {c} invalide.")
            self.cartes = self.custom_cards
        else : #52 cartes par défaut
            self.cartes = []
            for couleur in Carte.COULEURS :
                for valeur in Carte.VALEURS :
                    self.cartes.append(Carte(valeur, couleur).cacher())


    def melanger(self) :
        if self.ui : print("Mélange du deck")
        random.shuffle(self.cartes)

    def montrer(self) :
        for c in self.cartes :
            c.montrer()
        return self

    def cacher(self) :
        for c in self.cartes :
            c.cacher()
        return self

    def tirer(self) : #top
        if self.ui : print("Pioche d'une carte du deck")
        return self.cartes.pop(0) if self.cartes else None

    def ajouter(self, carte) : #bottom
        if self.ui : print("Ajout d'une carte au fond du deck")
        self.cartes.append(carte)

In [4]:
class Main :
    """ La main d'un joueur.
    """
    def __init__(self, deck, nb_cartes) :
        self.cartes = None
        self.selected = [] #indice des cartes à montrer
        self.piocher(deck, nb_cartes)

    def __str__(self) :
        affichage = ""
        for i in range(len(self.cartes)) :
            if (i+1) in self.selected :
                affichage += self.cartes[i].montrer().__str__()+" "
            else :
                affichage += self.cartes[i].__str__()+" "
        return affichage

    def __len__(self) :
        return len(self.cartes)

    def piocher(self, deck, nb_cartes) :
        self.cartes = [deck.tirer() for _ in range(nb_cartes)]

    def select(self,i) : #à partir de 1 : indice de la carte choisie
        if i < 1 or i > len(self.cartes) :
            raise ValueError(f"Indice {i} invalide.")
        self.selected.append(i)
        return self.cartes[i-1]

    def deselect(self) :
        self.selected = []

    def show_all(self) :
        for c in self.cartes :
            c.montrer()
        return self

    def hide_all(self) :
        for c in self.cartes :
            c.cacher()
        return self

In [5]:
class Pyramide :
    """ Une pyramide de cartes.
    """
    def __init__(self, hauteur, deck) :
        if hauteur < 1 :
            raise ValueError(f"Hauteur {hauteur} invalide.")
        if len(deck.cartes) < (hauteur*(hauteur+1)//2) :
            raise ValueError(f"Pas assez de cartes dans le deck ({len(deck.cartes)} cartes).")

        self.hauteur = hauteur #nb cartes = h(h+1)/2
        self.cartes = []
        self.piocher(deck)
        self.icartesamontrer = []
        self.icartesacacher = []
        self.cacher_tout()

    def __str__(self):
        affichage = ""; k=0
        for etage in range(self.hauteur+1) :
            affichage += "   " * (self.hauteur-etage)
            for j in range(k, k+etage) :
                affichage += self.cartes[j].__str__()+" "
            k += etage
            affichage += "\n"
        return affichage

    def piocher(self, deck) : #private
        for i in range(self.hauteur*(self.hauteur+1)//2) :
            c = deck.tirer()
            self.cartes.append(c)

    def generer_indices_ordre(self) : #private
        """ordre des cartes à retourner pendant une  partie"""
        self.icartesamontrer, self.icartesacacher = [], []

        for etage in range(self.hauteur) :
            i = (etage*(etage+1))//2
            self.icartesamontrer[:0] = range(i,i+etage+1)#...[3,4,5] [1,2] [0]



    def montrer_prochaine_carte(self) :
        i = self.icartesamontrer.pop(0) #enleve le premier elmt
        self.icartesacacher.append(i)   #l'ajoute ici à la fin
        self.cartes[i].montrer()
        return self.cartes[i]

    def montrer_tout(self) :
        self.generer_indices_ordre()
        self.icartesamontrer, self.icartesacacher = self.icartesacacher, self.icartesamontrer

        for c in self.cartes :
            c.montrer()
        return self

    def cacher_tout(self) :
        self.generer_indices_ordre()
        for c in self.cartes :
            c.cacher()
        return self

### **Classes : différents Joueurs**

In [6]:
class Joueur:
    """ Un joueur HUMAIN.
    """
    ACTIONS = {"o" : "oui", "n" : "non"}
    STOP = ["stop", "quit", "q", "exit"]

    def __init__(self, nom, human=True, ui=True):
        self.nom = nom
        self.human = human
        self.ui = ui #affichage des questions

        self.stop_game = False
        self.main = None
        self.points = 0

    def __str__(self):
        """ Affichage """
        return f"{self.nom} ({self.points} pts) : {self.main.__str__()}"

    def show_stats(self):
        """ Affichage """
        print(f"{self.nom} (Human) : {self.points} pts")


    # actions de début de jeu
    def recevoir_main(self, deck, nb_cartes):
        self.main = Main(deck, nb_cartes)

    # actions pendant un tour
    def choose_action1(self, carte) :
        """ Choisir entre jouer une carte ou passer son tour."""
        i=None
        while i not in self.ACTIONS :
            i = input()
            if i in self.STOP :
                self.stop_game = True
                return False
        return i

    def choose_action2(self, carte) :
        """ Choisir entre dénoncer l'adversaire ou le laisser."""
        i=None
        while i not in self.ACTIONS :
            i = input()
            if i in self.STOP :
                self.stop_game = True
                return False
        return i

    def possede_carte(self, carte):
        """ return indice de la carte si elle est dans la main
        [ 1 ] [ 2 ] [ 3 ] [ 4 ].... ou False (0)"""
        i=1
        for c in self.main.cartes :
            if c.same_value(carte) :
                return i
            i+=1
        return False

    # actions de fin de tour
    def ajouter_points(self, points) :
        self.points += points
        return self.points

    def claimed(self, points, success, bluff, accused, manche) : #le joueur a claim une carte...
        if not success :       #...et il n'a pas gagné
            self.ajouter_points(points)

    def denoncer(self, points, success, card, manche) : #le joueur a dénoncé l'adversaire
        if not success :       #...et l'adversaire n'avait pas bluffé
            self.ajouter_points(points)

    def not_denoncer(self, points, manche) :  #le joueur n'a pas dénoncé l'adversaire
        self.ajouter_points(points)

In [7]:
class JoueurRandom(Joueur):
    """ Joueur qui joue au hasard avec une proba uniforme à chaque action possible.
    """
    def __init__(self, nom):
        super().__init__(nom, human=False, ui=False)    #attributs parent : nom, human, ui, stop_game, main, points

    def show_stats(self):
        """ Affichage """
        print(f"{self.nom} (Random) : {self.points} pts")

    def choose_action1(self, carte) :
        """ Choisir entre jouer une carte ou passer son tour."""
        return "o" if random.random() < 0.5 else "n"

    def choose_action2(self, carte) :
        """ Choisir entre dénoncer l'adversaire ou le laisser."""
        return "o" if random.random() < 0.5 else "n"

In [8]:
class JoueurPresqueRandom(Joueur):
    """ Joueur qui joue avec des probas fixées.
    """
    def __init__(self, nom, probaBluff=0.2, probaDenonce=0.5):
        super().__init__(nom, human=False, ui=False)    #attributs parent : nom, human, ui, stop_game, main, points
        self.probaBluff = probaBluff                    # Probabilité de bluff du joueur
        self.probaDenonce = probaDenonce                # Probabilité de dénonciation du joueur

    # méthodes parent :
    # recevoir_main(self, deck, nb_cartes) - show_stats()
    # choose_action1(self) - choose_action2(self)
    # possede_carte(self, carte) - ajouter_points(self, points)
    # claimed(self, points, success, bluff, accused, manche) - denoncer(self, points, success, card, manche) - not_denoncer(self, points, manche)

    def show_stats(self):
        """ Affichage """
        print(f"{self.nom} (IA Simple) : {self.points} pts")
        print(f"Proba de bluffer     : {self.probaBluff}")
        print(f"Proba de dénoncement : {self.probaDenonce}")


    def choose_action1(self, carte) : # Le joueur bluffe s'il a une probabilité de bluff > une valeur aléatoire ou s'il possède la carte
        """ Choisir entre jouer une carte ou passer son tour."""
        return "o" if random.random() < self.probaBluff or self.possede_carte(carte) else "n"

    def choose_action2(self, carte) :
        """ Choisir entre dénoncer l'adversaire ou le laisser."""
        return "o" if random.random() < self.probaDenonce else "n"


In [9]:
class AdversaireIA(JoueurPresqueRandom):
    """ Joueur qui s'adapte en fonction des actions de l'adversaire.
    """
    def __init__(self, nom="IA", probaBluff=0.2, probaDenonce=0.2):
        super().__init__(nom, probaBluff, probaDenonce)  #attributs parent : nom, human, ui, stop_game, main, points - probaBluff, probaDenonce

        self.cartesAdversaire = []  # Mémorise les cartes de l'adversaire
        self.bluffReussis = 0
        self.bluffRate = 0
        self.denonceReussi = 0
        self.denonceRate = 0
        #self.cartesRetournees = []
        #self.cartesInterditesBluff = []
        self.cptAdvClaim  = 0       # Le nombre de fois que l'adversaire a joué une carte
        self.advBluffeur = False
        self.cptAccuse = 0          # Le nombre de fois que l'adversaire a accusé
        self.stopMentir = False

    # méthodes parent :
    # recevoir_main(self, deck, nb_cartes) - show_stats()
    # choose_action1(self) - choose_action2(self)
    # possede_carte(self, carte) - ajouter_points(self, points)
    # claimed(self, points, success, bluff, accused, manche) - denoncer(self, points, success, card, manche) - not_denoncer(self, points, manche)

    def show_stats(self):
        print(f"{self.nom} (IA Améliorée) : {self.points} pts")
        print(f"Proba de bluffer              : {self.probaBluff}")
        print(f"Proba de dénoncement          : {self.probaDenonce}")
        print(f"Bluff réussis/ratés           : {self.bluffReussis} | {self.bluffRate}")
        print(f"Dénonciations reussies/ratées : {self.denonceReussi} | {self.denonceRate}")
        print(f"Détection bluff abusif        : {self.advBluffeur} - {self.cptAdvClaim}")
        print(f"Nombre d'accusations reçues   : {self.cptAccuse}")
        print("Cartes mémorisées : ", end=' ')
        for c in self.cartesAdversaire : print(c, end=' ')
        print('\n')


    # actions pendant un tour
    def choose_action2(self, carte):
        """ Choisir entre dénoncer l'adversaire ou le laisser."""
        for c in self.cartesAdversaire :                    # Si la carte est connue comme appartenant au joueur, ne pas accuser
            if c.same_value(carte) :
                return "n"
        if len(self.cartesAdversaire) >= len(self.main) :   # Si l'IA connaît toutes les cartes du joueur, elle accuse systématiquement un bluff
            return "o"
        return super().choose_action2(carte)                # Sinon, comportement normal avec probabilité



    # actions de fin de tour :
    # répartition des points et recalculs en fonction de si le joueur a joué une carte/dénoncé/pas dénoncé
    def claimed(self, points, success, bluff, accused, manche) : #le joueur a claim une carte...
        if success :    #...et il a gagné
            if bluff :
                self.bluffReussis += 1          # L'adversaire n'a pas dénoncé
                self.recalcul_proba(manche)
                return
            else :                              # Le joueur dit la vérité, l'adversaire a dénoncé ou n'a pas dénoncé
                if accused :
                    self.cptAccuse += 1
                return
        else :          #...et il n'a pas gagné
            self.cptAccuse += 1
            self.ajouter_points(points)         # Le bluffeur est puni (2 points)
            self.bluffRate += 1                 # L'IA enregistre une bluff ratée
            self.recalcul_proba(manche)               # on pourrait décider de moins bluffer ici
            return

    def denoncer(self, points, success, card, manche) : #le joueur a dénoncé l'adversaire
        self.cptAdvClaim += 1
        if success :    #...et l'adversaire avait bluffé
            self.denonceReussi += 1             # L'IA enregistre une dénonciation réussie
            self.recalcul_proba(manche)
            return
        else :          #...et l'adversaire n'avait pas bluffé
            self.ajouter_points(points)         # L'IA est punie fortement pour une accusation erronée (3 points)

            self.denonceRate += 1               # L'IA enregistre une dénonciation ratée
            self.recalcul_proba(manche)               # on pourrait décider de moins dénoncer ici
            self.cartesAdversaire.append(card)  # L'IA mémorise la carte correcte du joueur
            return

    def not_denoncer(self, points, manche) : #le joueur n'a pas dénoncé l'adversaire
        self.ajouter_points(points)             # Pas contesté (1 point)
        self.cptAdvClaim += 1
        self.recalcul_proba(manche)
        return

    def not_denoncer(self, points, manche) : #le joueur n'a pas dénoncé l'adversaire
        self.ajouter_points(points)             # Pas contesté (1 point)
        self.cptAdvClaim += 1
        self.recalcul_proba(manche)
        return




    def recalcul_proba(self, manche):
        """ Recalcul des probabilités en fonction des actions de la manche en cours
        """
        # -- v3 -- #
        # Si l'adversaire joue plus de cartes différentes qu'il n'a en main (c'est qu'il bluff sûrement), l'IA va plus souvent le dénoncer
        if self.cptAdvClaim >= len(self.main) and not self.advBluffeur :
            self.probaDenonce = min(self.probaDenonce+0.5, 0.9)
            self.advBluffeur = True
        # Si l'adversaire dénonce souvent (>= 0.6), l'IA cesse presque totalement de bluffer
        if self.cptAccuse >= 10 and manche == 15 :
            self.stopMentir = True
            self.probaBluff = 0.01

        # Par défaut, l'IA adapte sa proba de dénoncer en fonction du nombre d'accusations réussies
        if not self.advBluffeur :
            self.probaDenonce = min(0.5, 0.2 + (self.denonceReussi - self.denonceRate) * 0.05)

        # et adapte sa proba de bluffer en fonction du nombre de bluff réussis
        if not self.stopMentir :
            self.probaBluff = max(0.01, self.probaBluff + (self.bluffReussis - self.bluffRate) * 0.05)


    """
    def recalcul_proba_v2(self, manche):
        # -- v2 -- #
        # Si l'adversaire dénonce souvent (>= 0.3), l'IA cesse presque totalement de bluffer
        if (self.cptAccuse >= 1 and manche == 20):
            self.stopMentir = True
            self.probaBluff = 0.01
        elif(self.stopMentir):
            self.probaBluff = 0.01
        else:
            self.probaBluff = max(0.01, self.probaBluff + (self.bluffReussis - self.bluffRate) * 0.05)

            self.probaDenonce = min(0.5, 0.2 + (self.denonceReussi - self.denonceRate) * 0.05)
    """

    """
    def recalcul_proba_v1(self, manche, adv_p_denonce):
        # -- v1 -- #
        # Si l'adversaire dénonce souvent (>= 0.3), l'IA cesse presque totalement de bluffer
        if adv_p_denonce >= 0.30:
            self.probaBluff = 0.01
        else:
            self.probaBluff = max(0.01, self.probaBluff + (self.bluffReussis - self.bluffRate) * 0.05)
        self.probaDenonce = min(0.5, 0.2 + (self.denonceReussi - self.denonceRate) * 0.05)
    """


### **Classe : Jeu**

In [ ]:
class Game:
    POINTS = {"claim" : 1, "denoncer_success" : 2, "denoncer_failed" : 3}

    def __init__(self, joueurs, nb_cartes_par_joueur=5, taille_pyramide=7, custom_deck=False, ui=True, desactiver_questions_h=False, desactiver_questions_ia=True, clear_between_rounds=True):
        self.ui = ui #affichage pyramide, mains etc
        self.desactiver_questions_h = desactiver_questions_h
        self.desactiver_questions_ia = desactiver_questions_ia
        self.clear_between_rounds = clear_between_rounds

        self.joueurs = joueurs
        self.nb_cartes_par_joueur = nb_cartes_par_joueur
        self.taille_pyramide = taille_pyramide
        self.deck = custom_deck if custom_deck else self.new_deck()
        self.deck.melanger()

        self.pyramide = None
        self.tour = 1
        self.inputs = {joueur.nom : False for joueur in self.joueurs}

        self.setUI()

    def __str__(self):
        affichage = ""
        affichage += self.pyramide.__str__()
        affichage += "\n"
        for joueur in self.joueurs :
            affichage += f"{joueur.__str__()}\n"
        return affichage

    def setUI(self) :
        if not self.ui :
            self.desactiver_questions_h = True
            self.desactiver_questions_ia = True
        if self.desactiver_questions_h and self.desactiver_questions_ia :
            self.ui=False

        for joueur in self.joueurs :
            if self.desactiver_questions_h and joueur.human :
                joueur.ui=False
            if self.desactiver_questions_ia and not joueur.human :
                joueur.ui=False

    def clearUI(self) :
        if self.clear_between_rounds :
            clear_output() #(si c'est un .ipynb)         #COMMENTEZ CETTE LIGNE SI ELLE POSE PROBLEME
            #os.system('cls||clear') #(si c'est un .py)
            return

    def new_deck(self) :
        deck = Deck()
        return deck

    def adversaire(self, joueur) :
        if joueur not in self.joueurs :
            raise ValueError("L'adversaire n'existe pas.")
        for j2 in self.joueurs :
            if j2 != joueur :
                return j2

    def winner(self) :
        """Le gagnant est celui qui a le moins de points"""
        jminpoints = 1000000000000
        jmin = None
        for joueur in self.joueurs :
            if joueur.points < jminpoints :
                jminpoints = joueur.points
                jmin = joueur
            elif joueur.points == jminpoints :
                jmin = None
        return jmin

    def not_denoncer(self, joueur, bluff) :
        p = self.POINTS["claim"]
        if self.ui : print(f"{joueur.nom} ne dénonce pas. {joueur.nom} prends {p} points")
        joueur.not_denoncer(p, manche=self.tour)                                                                         # not denounce
        self.adversaire(joueur).claimed(p, success=True, bluff=bluff, accused=False, manche=self.tour) #claimed success (avec ou sans bluff)

    def denoncer_success(self, joueur, carte_pyr) :
        p = self.POINTS["denoncer_success"]
        if self.ui : print(f"{self.adversaire(joueur).nom} a bluffé ! {self.adversaire(joueur).nom} prends {p} points")
        joueur.denoncer(p, success=True, card=carte_pyr, manche=self.tour)                             #denoncer success
        self.adversaire(joueur).claimed(p, success=False, bluff=True, accused=True, manche=self.tour)  #claimed failed (bluff raté)

    def denoncer_failed(self, joueur, carte_pyr) :
        p = self.POINTS["denoncer_failed"]
        self.adversaire(joueur).main.select(self.adversaire(joueur).possede_carte(carte_pyr))
        if self.ui :
            print(self.adversaire(joueur))
            print(f"{self.adversaire(joueur).nom} possédait la carte... {joueur.nom} prends {p} points")
        joueur.denoncer(p, success=False, card=carte_pyr, manche=self.tour)                             #denoncer failed
        self.adversaire(joueur).claimed(p, success=True, bluff=False, accused=True, manche=self.tour)   #claim success (pas bluff)



    # ---- jeu ---- #

    def jouer(self) : #loop jusque pyramide vide
        #-- début de partie : distribution --#
        self.pyramide = Pyramide(self.taille_pyramide, self.deck)
        for joueur in self.joueurs :
            joueur.recevoir_main(self.deck, self.nb_cartes_par_joueur)
            #afficher main de maniere super discrete
            if joueur.ui :
                joueur.main.show_all()
                input(f"Appuyez sur entrée pour voir la main de {joueur.nom}")
                print(joueur)
                input("Appuyez sur entrée pour continuer.")
                self.clearUI()
            joueur.main.hide_all()


        #-- tours : --#
        stop_game = False
        while self.pyramide.icartesamontrer and not stop_game :
            if self.ui : self.clearUI()
            # phase de début : une carte de la pyramide est retournée #

            carte_pyr = self.pyramide.montrer_prochaine_carte()
            if self.ui :
                print(self)
                print(f"\nVoici la nouvelle carte : {carte_pyr}\n")

            # phase d'action 1 : les joueurs choisissent s'ils jouent #

            for joueur in self.joueurs:
                self.inputs[joueur.nom]=False
            hastoplay=[]

            for joueur in self.joueurs :
                if joueur.ui :
                    print(f"----- {joueur.nom} -----")
                    print("Jouer une carte ?\n[ o ] [ n ]") #[print(f"[ {a} ] : {joueur.ACTIONS[a]}") for a in joueur.ACTIONS]
                i = joueur.choose_action1(carte_pyr)

                if not i :
                    stop_game = True
                    break #un joueur a demandé à arrêter la partie

                elif i=="o":
                    self.inputs[joueur.nom] = True
                    hastoplay.append(self.adversaire(joueur))

            if stop_game : break

            if self.ui :
                played=False;
                for joueur in self.joueurs :
                    if self.inputs[joueur.nom] :
                        print(f"Joueur {joueur.nom} a réclamé la carte."); played=True
                if not played :
                    print("Personne n'a joué. Tour suivant.")

            for joueur in self.joueurs:
                self.inputs[joueur.nom]=False

            # phase d'action 2 : les joueurs dénoncent éventuellement #

            for joueur in hastoplay :
                # dénoncer ?
                if joueur.ui :
                    print(f"----- {joueur.nom} -----")
                    print(f"Dénoncer l'adversaire {self.adversaire(joueur).nom} ?\n[ o ] [ n ]")

                i = joueur.choose_action2(carte_pyr)
                hascard = self.adversaire(joueur).possede_carte(carte_pyr)

                if not i :
                    stop_game = True
                    break

                elif i=="o": # a dénoncé
                    if self.ui : print(f"{joueur.nom} dénonce {self.adversaire(joueur).nom} !!!")

                    if hascard :
                        self.denoncer_failed(joueur, carte_pyr)  # POINT MALUS POUR joueur + révéler la carte dans la main
                    else :
                        self.denoncer_success(joueur, carte_pyr) # POINT MALUS POUR self.adversaire(joueur)

                elif i=="n" : # n'a pas dénoncé : POINT MALUS POUR joueur
                    if hascard :
                        self.not_denoncer(joueur, True) #bluff
                    else :
                        self.not_denoncer(joueur, False)


            # phase finale : tour suivant #

            if self.ui : input("Appuyez sur entrée pour continuer.")
            self.tour += 1

        #-- fin de partie : return points --#
        if self.ui :
            print("Toutes les cartes ont été retournées.") if not stop_game else print("Fin de la partie.")
            [print(joueur) for joueur in self.joueurs]
        return self.winner()

## **3 - _Lancement du jeu_**
Executez de préférence les cellules suivantes une par une pour analyser une partie de votre choix, ou pour jouer.

##### Fonction qui lance des parties

In [11]:
def stats(j1, j2, n_parties=1000, nb_val=100, taille_pyramide=27, nb_cartes_par_joueur=4, savej1=False, savej2=False) :
    if not savej1 : j1_reset = deepcopy(j1)
    if not savej2 : j2_reset = deepcopy(j2)
    j1_wins = 0; j2_wins = 0

    for  _ in range(n_parties) :
        # reset
        cards   = [Carte(valeur, couleur, custom_val=True) for valeur in range(1, nb_val+1) for couleur in Carte.COULEURS]  # 4 cartes par valeur
        custom_deck = Deck(custom_cards=cards)
        if not savej1 : j1 = deepcopy(j1_reset)
        if not savej2 : j2 = deepcopy(j2_reset)

        # lancement de la partie
        game = Game(joueurs=[j1, j2], custom_deck=custom_deck, taille_pyramide=taille_pyramide, nb_cartes_par_joueur=nb_cartes_par_joueur, ui=False)
        winner = game.jouer()

        # points
        if winner == j1 :
            j1_wins += 1
        elif winner == j2 :
            j2_wins += 1

    return j1_wins, j2_wins, n_parties, j1, j2

##### Bot Simple vs IA : 400 cartes, 120 tours (15 étages), 20 cartes par joueur, pas d'affichage, 1 partie

In [48]:
#main

cards = [Carte(valeur, couleur, custom_val=True) for valeur in range(1, 100) for couleur in Carte.COULEURS]  # 4 cartes par valeur
custom_deck = Deck(custom_cards=cards)

j1 = JoueurPresqueRandom("Bertrand")
j2 = AdversaireIA("Caroline")

game = Game(joueurs=[j1, j2], custom_deck=custom_deck, taille_pyramide=15, nb_cartes_par_joueur=20, ui=False)
#game = Game(joueurs, ui=False, desactiver_questions_h=True)

winner = game.jouer()

if winner :
    print('Gagnant :', winner.nom, '\n')
else :
    print("Personne n'a gagné.")

print("Mains des joueurs à la fin de la partie :")
print(j1)
print(j2, '\n')

print("Stats des joueurs :")
j1.show_stats(); print()
j2.show_stats()

Gagnant : Caroline 

Mains des joueurs à la fin de la partie :
Bertrand (79 pts) : [37♣] [ x ] [ x ] [ x ] [48♥] [ x ] [ x ] [39♠] [84♣] [88♥] [40♣] [ x ] [74♣] [17♣] [66♠] [87♦] [ x ] [ x ] [ x ] [ x ] 
Caroline (65 pts) : [97♦] [72♠] [ x ] [ x ] [ x ] [ x ] [54♣] [ x ] [29♥] [68♦] [ x ] [ x ] [ x ] [ x ] [ x ] [ x ] [ x ] [ x ] [ x ] [ x ]  

Stats des joueurs :
Bertrand (IA Simple) : 79 pts
Proba de bluffer     : 0.2
Proba de dénoncement : 0.5

Caroline (IA Améliorée) : 65 pts
Proba de bluffer              : 0.01
Proba de dénoncement          : 0.9
Bluff réussis/ratés           : 9 | 9
Dénonciations reussies/ratées : 16 | 10
Détection bluff abusif        : True - 43
Nombre d'accusations reçues   : 16
Cartes mémorisées :  [66♦] [84♥] [17♥] [87♠] [39♦] [37♦] [48♠] [88♠] [40♥] [74♥] 



##### Bot Simple vs IA : 400 cartes, 378 tours (27 étages), 20 cartes par joueur, pas d'affichage, 1000 parties

In [14]:
j1 = JoueurPresqueRandom("Bertrand", probaBluff=0.2, probaDenonce=0.5)
j2 = AdversaireIA("Caroline")

j1_wins, ia_wins, n_parties, j1, j2 = stats(j1, j2, n_parties=1000, nb_val=100, taille_pyramide=27, nb_cartes_par_joueur=4)

print(f'j1_wins : {j1_wins}/{n_parties}')
print(f'ia_wins : {ia_wins}/{n_parties}')

j1_wins : 0/1000
ia_wins : 1000/1000


##### Joueur humain vs IA avancée : Partie Classique

Actions possibles : "o" ou "n" pour oui/non pour jouer. Sinon "q" ou "exit" pour quitter. Mémorisez bien vos cartes car vous ne les reverrez pas !
> Remarque : l'affichage fonctionne bien sur Google Colab (.ipynb) et sur Spyder (.py), mais peut éventuellement poser problème sur VSCode (.ipynb) selon les extensions

In [51]:
nom = input("Entrez votre nom : ")
game = Game(joueurs=[Joueur(nom), AdversaireIA("Isabella")])#, taille_pyramide=5, nb_cartes_par_joueur=1)
winner = game.jouer()

if winner :
    print('Gagnant :')
    print(winner.nom)
else :
    print("Egalité.")

                     
                  [2 ♦] 
               [5 ♠] [J ♠] 
            [K ♦] [10♠] [10♥] 
         [6 ♥] [A ♦] [2 ♣] [8 ♣] 
      [5 ♦] [7 ♣] [Q ♣] [3 ♠] [8 ♦] 
   [10♦] [3 ♦] [7 ♠] [2 ♥] [9 ♣] [9 ♠] 
[A ♣] [A ♥] [10♣] [Q ♦] [4 ♠] [K ♥] [8 ♥] 

human (24 pts) : [ x ] [3 ♥] [ x ] [ x ] [ x ] 
Isabella (23 pts) : [4 ♥] [J ♦] [5 ♥] [7 ♦] [K ♣] 


Voici la nouvelle carte : [2 ♦]

----- human -----
Jouer une carte ?
[ o ] [ n ]
o
Joueur human a réclamé la carte.
Isabella dénonce human !!!
human (24 pts) : [ x ] [3 ♥] [ x ] [2 ♠] [ x ] 
human possédait la carte... Isabella prends 3 points
Appuyez sur entrée pour continuer.
Toutes les cartes ont été retournées.
human (24 pts) : [ x ] [3 ♥] [ x ] [2 ♠] [ x ] 
Isabella (26 pts) : [4 ♥] [J ♦] [5 ♥] [7 ♦] [K ♣] 
Gagnant :
human


## **4 - _Notre IA_**

L'IA que l'on a choisit d'implémenter a la capacité d'adapter son style de jeu en fonction de la partie en cours. Le jeu permet de choisir entre bluffer et ne pas bluffer lorsqu'une carte de la pyramide est retournée. Par la suite c'est un choix entre dénoncer ou ne pas dénoncer son adversaire si celui-ci a joué. 

---

Chez notre IA, ces deux actions sont déterminées par des probabilités : *P_bluff* ainsi que *P_denonce*. Ce sont celles ci qui s'adaptent au cours de la partie.

Nous avons déterminé que les stratégies gagnantes sont de

- Augmenter ou diminuer *P_bluff* en fonction du nombre de bluff réussi
- De même avec *P_denonce*
- Augmenter *P_denonce* si l'adversaire enchaîne les bluffs
- Mémoriser les cartes jouées par l'adversaire pour détecter les bluff garantis

---

En comparant les parties jouées, on constate que l'IA est bien meilleure sur les parties très longues avec de grandes pyramides et un grand nombre de cartes. L'IA se sert de plusieurs manches du début pour s'entraîner. Malgré cela, ses capacités permettent d'avoir de très bonnes performances sur des parties courtes.
